In [2]:
#ALL OF THIS IS NOW LEGACY, TURNS OUT JOSH PETTIT ALREADY CLEANED THE POES DATA AND HAS THE DATA IN FLUX UNITS IN THE MPE DATASET
#NEED TO RESTART KERNEL TO LOAD CHANGES TO IMPORTS :(

import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../../src'))

import datetime
import matplotlib.pyplot as plt
import numpy as np
from cdflib.epochs_astropy import CDFAstropy as cdfepoch
import data_loader
import importlib
import pandas as pd

importlib.reload(data_loader)
%matplotlib qt

In [3]:
start = datetime.datetime(year = 2013, month=12, day=1)
end = datetime.datetime(year = 2013, month=12, day=2)

refs_LEGACY = data_loader.load_raw_data_from_config(id=["POES", "SEM", "L2"],
                                                    satellite = "m02",
                                                    start = start,
                                                    end = end)

refs_MODERN = data_loader.load_raw_data_from_config(id=["POES", "SEM", "L1B"],
                                                    satellite = "m02",
                                                    start = start,
                                                    end = end)

refs_LEGACY['EPOCH'] = cdfepoch.to_datetime(refs_LEGACY["EPOCH"])

valid_dates = np.isfinite(refs_MODERN['year']) & np.isfinite(refs_MODERN['day']) & np.isfinite(refs_MODERN['msec'])
refs_MODERN['year'] = refs_MODERN['year'][valid_dates]
refs_MODERN['day'] = refs_MODERN['day'][valid_dates]
refs_MODERN['msec'] = refs_MODERN['msec'][valid_dates]
        
refs_MODERN['EPOCH'] = np.array([datetime.datetime(year = refs_MODERN['year'][i], month = 1, day = 1, hour = 0, minute = 0, second = 0, microsecond = 0) + datetime.timedelta(days = int(refs_MODERN['day'][i]) - 1) + datetime.timedelta(milliseconds = int(refs_MODERN['msec'][i])) for i in range(len(refs_MODERN['year']))])
             

KeyError: 'EPOCH'

In [3]:
#Plotting raw counts

plt.semilogy(refs_LEGACY['EPOCH'], refs_LEGACY['mep0E'][:, 0], label="Channel 0")
plt.semilogy(refs_LEGACY['EPOCH'], refs_LEGACY['mep0E'][:, 1], label="Channel 1")
plt.semilogy(refs_LEGACY['EPOCH'], refs_LEGACY['mep0E'][:, 2], label="Channel 2")
plt.legend()
plt.ylabel("Counts")
plt.xlabel("Time")
plt.show()

In [4]:
fig, axs = plt.subplots(3, 1, sharex=True, sharey=False)

#Geometrical Factors

G0 = 100 / 1.24
G1 = 100 / 1.44

G0 = 100

print(refs_LEGACY['mep0E'][:, 0].shape)
print(refs_MODERN['mep_ele_tel0_flux_e1'].shape)

modern_flux = []
modern_epoch = []
for m in refs_LEGACY['EPOCH']:
    start_period = m - datetime.timedelta(seconds=8)
    end_period = m + datetime.timedelta(seconds=8)
    within_time_interval = (start_period <= refs_MODERN["EPOCH"]) & (refs_MODERN["EPOCH"] < end_period)
    modern_epoch.append(m)
    modern_flux.append(np.nanmean(refs_MODERN['mep_ele_tel0_flux_e1'][within_time_interval]))
        
legacy_flux = G0 * refs_LEGACY['mep0E'][:, 0]

modern_flux = np.asarray(modern_flux)

axs[0].semilogy(refs_LEGACY["EPOCH"], legacy_flux, label="Legacy")
axs[1].semilogy(modern_epoch, modern_flux, label="Modern")

percent_error = ((modern_flux-legacy_flux)/modern_flux) * 100

axs[2].plot(modern_epoch, percent_error)

axs[0].legend()
axs[1].legend()



plt.xlabel("Time")
plt.show()

(10800,)
(86398,)


C:\Users\brofe\AppData\Local\Temp\ipykernel_84492\2590649441.py:20: RuntimeWarning: Mean of empty slice
  modern_flux.append(np.nanmean(refs_MODERN['mep_ele_tel0_flux_e1'][within_time_interval]))
C:\Users\brofe\AppData\Local\Temp\ipykernel_84492\2590649441.py:29: RuntimeWarning: divide by zero encountered in divide
  percent_error = ((modern_flux-legacy_flux)/modern_flux) * 100
C:\Users\brofe\AppData\Local\Temp\ipykernel_84492\2590649441.py:29: RuntimeWarning: invalid value encountered in divide
  percent_error = ((modern_flux-legacy_flux)/modern_flux) * 100
